# Объемные признаки

- Скользящие средние и z-score объема
- Volume Profile: Point of Control (POC), Value Area (VA), width, position
- Spike-индексы (резкие всплески)

В этом ноутбуке реализована самая сложная часть - расчёт горизонтального профиля объема по окну.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns   
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("✅ Библиотеки загружены")


✅ Библиотеки загружены


## Функции расчета объемных признаков


In [3]:
def volume_ma(volume, window=20):
    """Скользящее среднее объема"""
    return volume.rolling(window=window).mean()


def volume_zscore(volume, window=60):
    """Z-score объема: нормализованное отклонение от среднего"""
    ma = volume.rolling(window=window).mean()
    std = volume.rolling(window=window).std()
    return (volume - ma) / std


def volume_spike(volume, threshold=2.0, window=20):
    """Индикатор всплеска объема: превышает ли объем порог в std"""
    zscore = volume_zscore(volume, window=window)
    return (zscore > threshold).astype(int)


def calculate_volume_profile(df, window=20, num_bins=50):
    """
    Расчет Volume Profile для скользящего окна
    Возвращает: POC (Point of Control), Value Area High, Value Area Low
    """
    poc_list = []
    va_high_list = []
    va_low_list = []
    
    for i in range(len(df)):
        if i < window:
            poc_list.append(np.nan)
            va_high_list.append(np.nan)
            va_low_list.append(np.nan)
            continue
        
        window_data = df.iloc[i-window:i]
        
        # Создаем ценовые уровни (bins)
        price_min = window_data['low'].min()
        price_max = window_data['high'].max()
        bins = np.linspace(price_min, price_max, num_bins)
        
        # Распределяем объем по ценовым уровням
        volume_by_price = np.zeros(len(bins)-1)
        for _, row in window_data.iterrows():
            # Находим bins, которые пересекаются с [low, high] этого бара
            mask = (bins[:-1] >= row['low']) & (bins[1:] <= row['high'])
            volume_by_price[mask] += row['volume'] / mask.sum() if mask.sum() > 0 else 0
        
        # POC: уровень с максимальным объемом
        poc_idx = volume_by_price.argmax()
        poc = (bins[poc_idx] + bins[poc_idx+1]) / 2
        poc_list.append(poc)
        
        # Value Area: 70% объема вокруг POC
        total_volume = volume_by_price.sum()
        target_volume = total_volume * 0.70
        
        # Расширяем от POC вверх и вниз
        va_volume = volume_by_price[poc_idx]
        va_low_idx = poc_idx
        va_high_idx = poc_idx
        
        while va_volume < target_volume and (va_low_idx > 0 or va_high_idx < len(volume_by_price)-1):
            if va_low_idx > 0:
                va_volume += volume_by_price[va_low_idx-1]
                va_low_idx -= 1
            if va_high_idx < len(volume_by_price)-1 and va_volume < target_volume:
                va_volume += volume_by_price[va_high_idx+1]
                va_high_idx += 1
        
        va_low = bins[va_low_idx]
        va_high = bins[va_high_idx+1]
        
        va_high_list.append(va_high)
        va_low_list.append(va_low)
    
    return pd.Series(poc_list, index=df.index), pd.Series(va_high_list, index=df.index), pd.Series(va_low_list, index=df.index)

print("✅ Функции объема загружены")


✅ Функции объема загружены


## Загрузка, расчет и сохранение признаков


In [4]:
# Получаем текущую рабочую директорию
current_dir = Path.cwd()
print(f"Текущая директория: {current_dir}")

# Строим путь к данным относительно текущей директории
# Если ноутбук в 'ML/02_feature_engineering/', а данные в 'ML/data/processed'
DATA_DIR = current_dir.parent.parent / "ML" / "data" / "processed"
OUTPUT_DIR = Path('data') / 'features'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
print(f"Путь к данным: {DATA_DIR}")

# Проверяем существование пути
if DATA_DIR.exists():
    print("Директория найдена!")
else:
    print("Директория не найдена, проверьте путь")

# Список всех доступных тикеров включая индекс MOEX
tickers = [
    'AFKS', 'AFLT', 'ALRS', 'BELU', 'BSPB', 'CHMF', 'FIVE', 'GAZP', 'GMKN', 'HYDR',
    'IMOEX',  # Индекс Мосбиржи
    'IRAO', 'LENT', 'LKOH', 'MAGN', 'MGNT', 'MTSS', 'NLMK', 'NVTK', 'OZON',
    'PIKK', 'PLZL', 'ROSN', 'RTKM', 'SBER', 'SNGS', 'TATN', 'TCSG', 'VKCO', 'VTBR', 'YNDX'
]

print(f"✅ Загрузим {len(tickers)} тикеров")
print(f"Тикеры: {tickers}")

# Загружаем все данные в словарь
processed_data = {}
for ticker in tickers:
    try:
        print(f"🔄 Обработка {ticker}...", end=" ")
        
        # 1. Загрузка данных
        input_path = DATA_DIR / f"{ticker}_ohlcv_returns.parquet"
        if not input_path.exists():
            print(f"⚠️ Файл не найден: {input_path}")
            continue
            
        df = pd.read_parquet(input_path)
        
        # 2. Расчет объемных признаков
        # Скользящие средние и Z-score
        df['volume_ma_20'] = volume_ma(df['volume'], window=20)
        df['volume_ma_60'] = volume_ma(df['volume'], window=60)
        df['volume_zscore'] = volume_zscore(df['volume'], window=60)
        
        # Индикатор всплеска объема
        df['volume_spike'] = volume_spike(df['volume'], threshold=2.0, window=20)
        
        # Volume Profile (самая ресурсоемкая часть)
        # Если будет слишком долго, можно временно закомментировать для теста
        df['vp_poc'], df['vp_va_high'], df['vp_va_low'] = calculate_volume_profile(df, window=20, num_bins=50)
        
        # Производные от профиля объема
        df['vp_width'] = df['vp_va_high'] - df['vp_va_low']
        
        # Позиция цены относительно профиля (нормированная)
        # Избегаем деления на ноль, если ширина равна 0
        df['vp_position'] = np.where(
            df['vp_width'] != 0,
            (df['close'] - df['vp_poc']) / df['vp_width'],
            0
        )

        # 3. Сохранение
        output_path = OUTPUT_DIR / f"{ticker}_volume_features.parquet"
        df.to_parquet(output_path, index=False)
        
        processed_data[ticker] = len(df)
        print(f"✅ Готово ({len(df)} строк)")

    except Exception as e:
        print(f"\n❌ Ошибка при обработке {ticker}: {e}")

print("-" * 50)
print(f"🎉 Обработка завершена. Успешно обработано: {len(processed_data)} из {len(tickers)}")

Текущая директория: e:\Python\VolatilityChecker\MOEXScanner\ML\02_feature_engineering
Путь к данным: e:\Python\VolatilityChecker\MOEXScanner\ML\data\processed
Директория найдена!
✅ Загрузим 31 тикеров
Тикеры: ['AFKS', 'AFLT', 'ALRS', 'BELU', 'BSPB', 'CHMF', 'FIVE', 'GAZP', 'GMKN', 'HYDR', 'IMOEX', 'IRAO', 'LENT', 'LKOH', 'MAGN', 'MGNT', 'MTSS', 'NLMK', 'NVTK', 'OZON', 'PIKK', 'PLZL', 'ROSN', 'RTKM', 'SBER', 'SNGS', 'TATN', 'TCSG', 'VKCO', 'VTBR', 'YNDX']
🔄 Обработка AFKS... ✅ Готово (1301 строк)
🔄 Обработка AFLT... ✅ Готово (1301 строк)
🔄 Обработка ALRS... ✅ Готово (1301 строк)
🔄 Обработка BELU... ✅ Готово (1289 строк)
🔄 Обработка BSPB... ✅ Готово (1295 строк)
🔄 Обработка CHMF... ✅ Готово (1301 строк)
🔄 Обработка FIVE... ✅ Готово (242 строк)
🔄 Обработка GAZP... ✅ Готово (1301 строк)
🔄 Обработка GMKN... ✅ Готово (1297 строк)
🔄 Обработка HYDR... ✅ Готово (1301 строк)
🔄 Обработка IMOEX... ✅ Готово (1253 строк)
🔄 Обработка IRAO... ✅ Готово (1299 строк)
🔄 Обработка LENT... ✅ Готово (990 стр